In [ ]:
   def crop_and_watch_lure(self, res):
        print("Watching lure")
        watch_time = time.time()

        while True:
            print("still looking...")
            try: 
                # Define the res
                x, y, w, h = res 
                # Crop the image
                image = self.main_agent.cur_img[(y - SHIFT):y+(h + SHIFT), (x - SHIFT):x +(w + SHIFT)]

                # Convert the image to hsv
                image_hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
                lower_yellow = np.array([14, 75, 207])
                upper_yellow = np.array([34, 107, 227])

                # Define the mask for the threshold
                mask = cv.inRange(image_hsv, lower_yellow, upper_yellow)

                # Apply the mask using the and operation
                yellow_regions = cv.bitwise_and(image_hsv, image_hsv, mask=mask)

                if (yellow_regions != 0).any():
                    print("Fish hooked!")
                    self.pull_line()
                if time.time() - watch_time >= FISHING_WATCH_TIME:
                    print("Time out")
                    break
                time.sleep(1)
            except:
                print("error")
        self.pull_line()